In [6]:
import pandas as pd
import numpy as np

In [15]:
done_bigrams=list(set(done_files))
fivegram_list='0 1 2 3 4 5 6 7 8 9 a_ aa ab ac ad ae af ag ah ai aj ak al am an ao ap aq ar as at au av aw ax ay az b_ ba bb bc bd be bf bg bh bi bj bk bl bm bn bo bp bq br bs bt bu bv bw bx by bz c_ ca cb cc cd ce cf cg ch ci cj ck cl cm cn co cp cq cr cs ct cu cv cw cx cy cz d_ da db dc dd de df dg dh di dj dk dl dm dn do dp dq dr ds dt du dv dw dx dy dz e_ ea eb ec ed ee ef eg eh ei ej ek el em en eo ep eq er es et eu ev ew ex ey ez f_ fa fb fc fd fe ff fg fh fi fj fk fl fm fn fo fp fq fr fs ft fu fv fw fx fy fz g_ ga gb gc gd ge gf gg gh gi gj gk gl gm gn go gp gq gr gs gt gu gv gw gx gy gz h_ ha hb hc hd he hf hg hh hi hj hk hl hm hn ho hp hq hr hs ht hu hv hw hx hy hz i_ ia ib ic id ie if ig ih ii ij ik il im in io ip iq ir is it iu iv iw ix iy iz j_ ja jb jc jd je jf jg jh ji jj jk jl jm jn jo jp jq jr js jt ju jv jw jx jy jz k_ ka kb kc kd ke kf kg kh ki kj kk kl km kn ko kp kq kr ks kt ku kv kw kx ky kz l_ la lb lc ld le lf lg lh li lj lk ll lm ln lo lp lq lr ls lt lu lv lw lx ly lz m_ ma mb mc md me mf mg mh mi mj mk ml mm mn mo mp mq mr ms mt mu mv mw mx my mz n_ na nb nc nd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt nu nv nw nx ny nz o_ oa ob oc od oe of og oh oi oj ok ol om on oo op oq or os ot other ou ov ow ox oy oz p_ pa pb pc pd pe pf pg ph pi pj pk pl pm pn po pp pq pr ps pt pu punctuation pv pw px py pz q_ qa qb qc qd qe qf qg qh qi qj ql qm qn qo qp qq qr qs qt qu qv qw qx qy qz r_ ra rb rc rd re rf rg rh ri rj rk rl rm rn ro rp rq rr rs rt ru rv rw rx ry rz s_ sa sb sc sd se sf sg sh si sj sk sl sm sn so sp sq sr ss st su sv sw sx sy sz t_ ta tb tc td te tf tg th ti tj tk tl tm tn to tp tq tr ts tt tu tv tw tx ty tz u_ ua ub uc ud ue uf ug uh ui uj uk ul um un uo up uq ur us ut uu uv uw ux uy uz v_ va vb vc vd ve vf vg vh vi vj vk vl vm vn vo vp vq vr vs vt vu vv vw vx vy vz w_ wa wb wc wd we wf wg wh wi wj wk wl wm wn wo wp wq wr ws wt wu wv ww wx wy wz x_ xa xb xc xd xe xf xg xh xi xj xk xl xm xn xo xp xq xr xs xt xu xv xw xx xy xz y_ ya yb yc yd ye yf yg yh yi yj yk yl ym yn yo yp yq yr ys yt yu yv yw yx yy yz z_ za zb zc zd ze zf zg zh zi zj zk zl zm zn zo zp zq zr zs zt zu zv zw zx zy zz'
fivegram_list=fivegram_list.split(" ")
to_do_list=set(fivegram_list).difference(done_bigrams)
done_list='0 1 2 3 4 5 6 7 8 9 other punctuation'
done_list=done_list.split(" ")
to_do_list.difference(done_list)

{'bq',
 'fq',
 'gq',
 'gz',
 'jq',
 'jz',
 'kq',
 'kz',
 'lq',
 'pz',
 'qf',
 'qg',
 'qh',
 'qj',
 'qm',
 'qq',
 'qy',
 'qz',
 'rq',
 'rz',
 'uq',
 'vq',
 'wq',
 'wz',
 'xb',
 'xf',
 'xk',
 'xq',
 'xw',
 'xz',
 'yj',
 'yq',
 'zc',
 'zf',
 'zg',
 'zj',
 'zk',
 'zq',
 'zt',
 'zz'}

In [6]:
df1=pd.read_pickle('/data/dharp/compounds/datasets/google/other1.pkl')

In [34]:
www_df=df.loc[df.lemma_sent.str.contains('web site')]

In [7]:
df1.loc[df1.comp_class!=0]

,lemma_sent,year,pos_sent,comp_class,count
2761,"£ , costa rica ,",1901,NOUN PUNCT NOUN NOUN PUNCT,4,1
2762,"£ , costa rica ,",1906,NOUN PUNCT NOUN NOUN PUNCT,4,1
2763,"£ , costa rica ,",1909,NOUN PUNCT NOUN NOUN PUNCT,4,2
2764,"£ , costa rica ,",1916,NOUN PUNCT NOUN NOUN PUNCT,4,12
2765,"£ , costa rica ,",1923,NOUN PUNCT NOUN NOUN PUNCT,4,2
...,...,...,...,...,...
265596,♦ butler 's handbook of,1909,NOUN NOUN PART NOUN ADP,2,1
265597,♦ butler 's handbook of,1910,NOUN NOUN PART NOUN ADP,2,9
265598,♦ butler 's handbook of,1911,NOUN NOUN PART NOUN ADP,2,16
265599,♦ butler 's handbook of,1912,NOUN NOUN PART NOUN ADP,2,19


In [36]:
www_df.year.value_counts().sort_index()

1801       2
1855       1
1886       1
1908       1
1914       1
1932       1
1942       2
1953       3
1954       1
1958       1
1959       1
1960       1
1961      12
1963       1
1964       1
1965       2
1967       4
1968      11
1969       1
1970       3
1971       2
1972      14
1973       4
1974       4
1975       5
1976       4
1977       4
1978      14
1979      33
1980      12
1981      23
1982      18
1983      34
1984      24
1985      13
1986      44
1987      64
1988      28
1989      47
1990      65
1991      65
1992      56
1993     112
1994     155
1995     517
1996    1224
1997    1677
1998    1904
1999    2040
2000    2144
2001    2218
2002    2220
2003    2234
2004    2242
2005    2210
2006    2214
2007    2204
2008    2160
Name: year, dtype: int64

In [37]:
www_df.loc[www_df.year<1900]

,lemma_sent,year,pos_sent,comp_class,count
35707800,web site : http://www.novapublishers.com the,1886,NOUN NOUN PUNCT X DET,2,1
35708668,web site at www .,1855,NOUN NOUN ADP NOUN PUNCT,2,1
35708899,"web site be , or",1801,NOUN NOUN AUX PUNCT CCONJ,2,3
56451798,web site refer to in,1801,NOUN NOUN VERB ADP ADP,2,3


In [38]:
www_df.loc[(www_df.year>=1900) & (www_df.year<1950)]

,lemma_sent,year,pos_sent,comp_class,count
15004884,"web site , and other",1942,NOUN NOUN PUNCT CCONJ ADJ,2,1
15007216,web site at www.albertwhitman.com .,1942,NOUN NOUN ADP X PUNCT,2,1
35706986,"web site , include one",1908,NOUN NOUN PUNCT VERB NUM,2,2
35707801,web site : http://www.novapublishers.com the,1914,NOUN NOUN PUNCT X DET,2,1
77182128,web site at www.schifferbooks.com we,1932,NOUN NOUN ADP X PRON,2,1


In [39]:
www_df.loc[(www_df.year>=1950) & (www_df.year<1970)]

,lemma_sent,year,pos_sent,comp_class,count
15004551,web site ) copyright ©,1960,NOUN NOUN PUNCT NOUN NOUN,1,1
15004552,web site ) copyright ©,1961,NOUN NOUN PUNCT NOUN NOUN,1,1
15004553,web site ) copyright ©,1963,NOUN NOUN PUNCT NOUN NOUN,1,1
15004554,web site ) copyright ©,1965,NOUN NOUN PUNCT NOUN NOUN,1,1
15004771,"web site , and in",1967,NOUN NOUN PUNCT CCONJ ADP,2,1
15005123,"web site , blog ,",1959,NOUN NOUN PUNCT NOUN PUNCT,2,1
15005124,"web site , blog ,",1967,NOUN NOUN PUNCT NOUN PUNCT,2,1
15005125,"web site , blog ,",1969,NOUN NOUN PUNCT NOUN PUNCT,2,1
15005985,"web site , which be",1968,NOUN NOUN PUNCT DET VERB,2,1
15006203,"web site , you need",1967,NOUN NOUN PUNCT PRON VERB,2,1


In [40]:
www_df.loc[(www_df.year>=1970) & (www_df.year<1980)]

,lemma_sent,year,pos_sent,comp_class,count
15004446,web site ( http://www.sec.gov ),1972,NOUN NOUN PUNCT NOUN PUNCT,2,1
15004555,web site ) copyright ©,1974,NOUN NOUN PUNCT NOUN NOUN,1,1
15005126,"web site , blog ,",1972,NOUN NOUN PUNCT NOUN PUNCT,2,1
15005341,"web site , get catch",1979,NOUN NOUN PUNCT AUX VERB,2,1
15005359,"web site , however ,",1972,NOUN NOUN PUNCT ADV PUNCT,2,2
...,...,...,...,...,...
77184732,web site or their content,1973,NOUN NOUN CCONJ PRON NOUN,2,1
77184733,web site or their content,1975,NOUN NOUN CCONJ PRON NOUN,2,1
77184734,web site or their content,1978,NOUN NOUN CCONJ PRON NOUN,2,1
77185066,web site that be accessible,1970,NOUN NOUN DET VERB ADJ,2,1


In [41]:
www_df.loc[(www_df.year>=1980) & (www_df.year<1990)]

,lemma_sent,year,pos_sent,comp_class,count
15004447,web site ( http://www.sec.gov ),1982,NOUN NOUN PUNCT NOUN PUNCT,2,3
15004556,web site ) copyright ©,1980,NOUN NOUN PUNCT NOUN NOUN,1,2
15004557,web site ) copyright ©,1984,NOUN NOUN PUNCT NOUN NOUN,1,1
15004612,"web site , a company",1989,NOUN NOUN PUNCT DET NOUN,2,2
15004680,"web site , and (",1987,NOUN NOUN PUNCT CCONJ PUNCT,2,1
...,...,...,...,...,...
77186063,web site www.oup.co.uk / vsi,1983,NOUN NOUN NOUN SYM NOUN,0,4
77186064,web site www.oup.co.uk / vsi,1986,NOUN NOUN NOUN SYM NOUN,0,2
77186065,web site www.oup.co.uk / vsi,1989,NOUN NOUN NOUN SYM NOUN,0,2
77186116,web site you feel should,1989,NOUN NOUN PRON VERB AUX,2,1


In [42]:
www_df.loc[(www_df.year>=1990) & (www_df.year<2000)]

,lemma_sent,year,pos_sent,comp_class,count
10862845,we have a web site,1998,PRON VERB DET NOUN NOUN,5,2
10862846,we have a web site,1999,PRON VERB DET NOUN NOUN,5,5
15000732,web page and web site,1997,NOUN NOUN CCONJ NOUN NOUN,1,2
15000733,web page and web site,1998,NOUN NOUN CCONJ NOUN NOUN,1,6
15000734,web page and web site,1999,NOUN NOUN CCONJ NOUN NOUN,1,4
...,...,...,...,...,...
79651205,well - organize web site,1999,ADV PUNCT VERB NOUN NOUN,5,17
80873367,well as the web site,1996,ADV ADP DET NOUN NOUN,5,1
80873368,well as the web site,1997,ADV ADP DET NOUN NOUN,5,7
80873369,well as the web site,1998,ADV ADP DET NOUN NOUN,5,3


In [6]:
tt.groupby(['modifier','head'])['count'].sum().sort_values().tail(40)

modifier    head       
common      prayer          470397
capital     stock           472644
catholic    university      476813
central     committee       483908
chief       justice         503431
american    revolution      521674
great       britain         536210
air         force           546698
central     government      571291
balance     sheet           574158
american    society         607238
dining      room            608095
east        coast           614320
dorsal      surface         617137
jesus       christ          627987
indian      affairs         628365
black       sea             645886
cultural    revolution      670382
dialog      box             689594
end         result          690635
common      pleas           693589
cold        war             706250
american    association     721372
british     isles           724410
            empire          918169
cross       section         958416
death       penalty         967603
attorney    general         989

In [2]:
df_path='/data/dharp/compounds/datasets/entire_df/'
df_files=[]
for filename in glob.glob(df_path+'*parq'):
    #print(filename)
    df_files.append(filename)

In [3]:
def ner_lemma_reducer(sent):
    ner_cat=None
    lemma=[]
    parsed_sent=nlp(sent)
    for token in parsed_sent:
        lemma.append(token.lemma_)
    lemma_sent=' '.join(lemma)
    
    if parsed_sent.ents:
        for e in parsed_sent.ents:
            #print(e.text,e.label_)
            if e.label_=='PERSON':
                ner_cat='PERSON'
    else:
        ner_cat=None
    #print(sent,ner_cat,lemma_sent)
    return pd.Series([ner_cat,lemma_sent])

In [4]:
br_to_us=pd.read_csv('data/spelling.csv',sep='\t')
br_to_us_dict=dict(zip(br_to_us.UK.tolist(),br_to_us.US.tolist()))


spelling_replacement={'context':br_to_us_dict,'modifier':br_to_us_dict,'head':br_to_us_dict,'word':br_to_us_dict}


any_word=r'.+_.+'
any_noun=r'.+_noun'
proper_noun=r'[a-z.-]+_noun'
space=r'\s'

In [5]:
ff=fastparquet.ParquetFile('/data/dharp/compounds/datasets/entire_df/df_4.parq')
current_df = next(iter(ff.iter_row_groups()))

In [6]:
len(ff.row_groups)

60

In [7]:
current_df.year=current_df.year.astype('int32')
df=current_df.head(100000).copy()
df=df.groupby(['fivegram_pos','year']).sum('count')

In [10]:
new_df=pd.DataFrame(df.index.get_level_values('fivegram_pos').unique().tolist())
new_df.columns=['old_index']
new_df[['l1_pos','word_pos','r1_pos','r2_pos','r3_pos']]=new_df['old_index'].str.split(space,expand=True,n=5)
new_df['sent']=new_df.l1_pos.str.split('_').str[0]+" "+new_df.word_pos.str.split('_').str[0]+" "+new_df.r1_pos.str.split('_').str[0]+" "+new_df.r2_pos.str.split('_').str[0]+" "+new_df.r3_pos.str.split('_').str[0]
new_df['pos']=new_df.l1_pos.str.split('_').str[1]+" "+new_df.word_pos.str.split('_').str[1]+" "+new_df.r1_pos.str.split('_').str[1]+" "+new_df.r2_pos.str.split('_').str[1]+" "+new_df.r3_pos.str.split('_').str[1]
#new_df=new_df[['old_index','sent','pos']]
new_df[['IS_PERSON','new_sent']]=new_df['sent'].apply(ner_lemma_reducer)#,axis=1)
new_df

,old_index,l1_pos,word_pos,r1_pos,r2_pos,r3_pos,sent,pos,IS_PERSON,new_sent
0,he'th_adp a_det overtheer_noun and_conj makin_...,he'th_adp,a_det,overtheer_noun,and_conj,makin_verb,he'th a overtheer and makin,adp det noun conj verb,None,he'th a overtheer and makin
1,he_adj being_noun dead_pron yet_verb speaketh_...,he_adj,being_noun,dead_pron,yet_verb,speaketh_noun,he being dead yet speaketh,adj noun pron verb noun,None,-PRON- be dead yet speaketh
2,he_adj has_noun taken_verb off_verb in_adp,he_adj,has_noun,taken_verb,off_verb,in_adp,he has taken off in,adj noun verb verb adp,None,-PRON- have take off in
3,he_adj purpose_noun of_adp this_det book_noun,he_adj,purpose_noun,of_adp,this_det,book_noun,he purpose of this book,adj noun adp det noun,None,-PRON- purpose of this book
4,he_pron '_. knows_verb '_pron that_adp,he_pron,'_.,knows_verb,'_pron,that_adp,he ' knows ' that,pron . verb pron adp,None,-PRON- ' know ' that
...,...,...,...,...,...,...,...,...,...,...
2891,he_pron accepted_verb the_det office_noun of_adp,he_pron,accepted_verb,the_det,office_noun,of_adp,he accepted the office of,pron verb det noun adp,None,-PRON- accept the office of
2892,he_pron accepted_verb the_det position_noun in...,he_pron,accepted_verb,the_det,position_noun,in_adp,he accepted the position in,pron verb det noun adp,None,-PRON- accept the position in
2893,he_pron accepted_verb the_det post_noun at_adp,he_pron,accepted_verb,the_det,post_noun,at_adp,he accepted the post at,pron verb det noun adp,None,-PRON- accept the post at
2894,he_pron accepted_verb the_det post_noun with_adp,he_pron,accepted_verb,the_det,post_noun,with_adp,he accepted the post with,pron verb det noun adp,None,-PRON- accept the post with


In [9]:
new_df.l1_pos

AttributeError: 'DataFrame' object has no attribute 'l1_pos'

In [128]:
new_df.pos.str.replace('(?!noun)',repl='other',regex=True)

0       otheraotherdotherpother otherdothereothertothe...
1       otheraotherdotherjother notherootheruothernoth...
2       otheraotherdotherjother notherootheruothernoth...
3       otheraotherdotherjother notherootheruothernoth...
4       otherpotherrotheroothernother other.other othe...
                              ...                        
2891    otherpotherrotheroothernother othervothereothe...
2892    otherpotherrotheroothernother othervothereothe...
2893    otherpotherrotheroothernother othervothereothe...
2894    otherpotherrotheroothernother othervothereothe...
2895    otherpotherrotheroothernother othervothereothe...
Name: pos, Length: 2896, dtype: object

In [97]:
[^ ]

SyntaxError: invalid syntax (<ipython-input-97-7788f1f472b7>, line 1)

In [9]:
' '.join(['a_','an','of','to','in','ad','wh','be','ha','is','co','wa','he','no','it','wi','fo','re','as','on','we','th','ma','pr','ar','ip','sh','ca','so','hi','bu','al','se','de','by','wo','st','fr','di','mo','su','at','or','yo','me','li','pa','do','ex','le','pe','po','if','ne','fi','un','fa','sa','ch','la','lo','ac','ho','mu','go','si','en','ev','tr'])

'a_ an of to in ad wh be ha is co wa he no it wi fo re as on we th ma pr ar ip sh ca so hi bu al se de by wo st fr di mo su at or yo me li pa do ex le pe po if ne fi un fa sa ch la lo ac ho mu go si en ev tr'

In [49]:
new_df.IS_PERSON.value_counts()

Series([], Name: IS_PERSON, dtype: int64)

In [14]:
doc=nlp(current_df.iloc[10].fivegram_pos)
for token in doc:
    print(token.text, token.pos_, token.dep_)

he'th_adp PROPN compound
a_det PROPN ROOT
overtheer_noun PROPN ROOT
and_conj PROPN compound
makin_verb PROPN ROOT
